In [83]:
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras import utils
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import gensim

In [84]:
data=open('corona.txt','r')
data=[text for text in data if text.count(' ')>=2]
data

['The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. \n',
 'Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission â€“transmission of the virus before the appearance of symptoms â€“ is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. \n',
 'The reproductive number â€“ the number of secondary infections generated from one infected individual â€“ is underst

In [85]:
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.text import Tokenizer
# vectorize=preprocessing.text.Tokenizer()
vectorize = Tokenizer()

In [86]:
vectorize.fit_on_texts(data)

In [87]:
data

['The speed of transmission is an important point of difference between the two viruses. Influenza has a shorter median incubation period (the time from infection to appearance of symptoms) and a shorter serial interval (the time between successive cases) than COVID-19 virus. The serial interval for COVID-19 virus is estimated to be 5-6 days, while for influenza virus, the serial interval is 3 days. This means that influenza can spread faster than COVID-19. \n',
 'Further, transmission in the first 3-5 days of illness, or potentially pre-symptomatic transmission â€“transmission of the virus before the appearance of symptoms â€“ is a major driver of transmission for influenza. In contrast, while we are learning that there are people who can shed COVID-19 virus 24-48 hours prior to symptom onset, at present, this does not appear to be a major driver of transmission. \n',
 'The reproductive number â€“ the number of secondary infections generated from one infected individual â€“ is underst

In [88]:
data=vectorize.texts_to_sequences(data)

In [89]:
data

[[1,
  38,
  2,
  8,
  9,
  39,
  40,
  41,
  2,
  42,
  13,
  1,
  43,
  23,
  3,
  44,
  11,
  24,
  45,
  46,
  47,
  1,
  14,
  25,
  48,
  10,
  26,
  2,
  27,
  12,
  11,
  24,
  15,
  16,
  1,
  14,
  13,
  49,
  50,
  17,
  4,
  5,
  6,
  1,
  15,
  16,
  7,
  4,
  5,
  6,
  9,
  51,
  10,
  18,
  19,
  52,
  20,
  28,
  7,
  3,
  6,
  1,
  15,
  16,
  9,
  29,
  20,
  30,
  53,
  31,
  3,
  32,
  54,
  55,
  17,
  4,
  5],
 [56,
  8,
  33,
  1,
  57,
  29,
  19,
  20,
  2,
  58,
  59,
  60,
  61,
  62,
  8,
  63,
  2,
  1,
  6,
  64,
  1,
  26,
  2,
  27,
  21,
  9,
  11,
  34,
  35,
  2,
  8,
  7,
  3,
  33,
  65,
  28,
  66,
  22,
  67,
  31,
  68,
  22,
  69,
  70,
  32,
  71,
  4,
  5,
  6,
  72,
  73,
  74,
  75,
  10,
  76,
  77,
  78,
  79,
  30,
  80,
  81,
  82,
  10,
  18,
  11,
  34,
  35,
  2,
  8],
 [1,
  83,
  36,
  21,
  1,
  36,
  2,
  84,
  85,
  86,
  25,
  87,
  88,
  89,
  21,
  9,
  90,
  10,
  18,
  13,
  37,
  12,
  37,
  19,
  7,
  4,
  5,
  6,
  91,
  

In [90]:
total_vocab = sum(len(s) for s in data)
word_count = len(vectorize.word_index) + 1
window_size = 2

In [91]:
def cbow_model(data, window_size, total_vocab):
    total_length = window_size*2
    for text in data:
        text_len = len(text)
        for idx, word in enumerate(text):
            context_word = []
            target   = []
            begin = idx - window_size
            end = idx + window_size + 1
            context_word.append([text[i] for i in range(begin, end) if 0 <= i < text_len and i != idx])
            target.append(word)
            
            contextual = sequence.pad_sequences(context_word,maxlen=total_length)
            final_target =utils.to_categorical(target, total_vocab)
            yield(contextual, final_target)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=total_vocab, output_dim=100, input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)))
model.add(Dense(total_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
for i in range(10):
    cost = 0
    for x, y in cbow_model(data, window_size, total_vocab):
        cost += model.train_on_batch(x, y)
    print(i, cost)

0 1041.4596967697144


In [ ]:
dims=100
vect_file=open('corona11.txt','w')
vect_file.write('{} {}\n'.format(total_vocab,dims))


In [ ]:
weights=model.get_weights()[0]
for text,i in vectorize.word_index.items():
    final=' '.join(map(str,list(weights[i,:])))
    vect_file.write('{} {}\n'.format(text,final))
vect_file.close()

In [ ]:
# cbow_output=gensim.models.KeyedVectors.load_word2vec_format('corona.txt',binary=False,limit=90)
# cbow_output.most_similar(positive=['virus'])
# cbow_output = gensim.models.KeyedVectors.load_word2vec_format('corona11.txt', binary=False, limit=90, encoding='utf-8')
# cbow_output.most_similar(positive=['virus'])

cbow_output = gensim.models.KeyedVectors.load_word2vec_format('corona11.txt', binary=False, limit=90, encoding='latin-1')
cbow_output.most_similar(positive=['virus'])